In [1]:
import pandas as pd
import pymongo
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# conn = psycopg2.connect(database="postgres", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
# conn.autocommit = True

In [ ]:
csv_file = "./Data/breweries.csv"
breweries_df = pd.read_csv(csv_file)
breweries_df.head()

In [ ]:
breweries_df['state'] = breweries_df['state'].str.replace(' ', '')

In [ ]:
breweries_df.head()

In [ ]:
breweries_df['state'].value_counts()

In [ ]:
breweries_df.replace(to_replace = "DC", value = "District of Columbia", inplace = True)
breweries_df['state'].value_counts()

In [ ]:
breweries_df = breweries_df.drop(columns=['Unnamed: 0', 'city']) 

In [ ]:
breweries_df = breweries_df.groupby(['state']).count()

In [ ]:
breweries_df.head(10)

In [ ]:
csv_file = "./Data/Costs of Excessive Alcohol Consumption by State.csv"
costs_df = pd.read_csv(csv_file)
costs_df.head()

In [ ]:
costs_df = costs_df.drop(columns=['Location']) 
costs_df.head()

In [ ]:
costs_df.rename(columns = {'State':'state', 'Total Cost ($)':'total_cost', 'Cost per drink ($)':'cost_per_drink',
                          'Cost per capita ($)':'cost_per_capita'}, inplace = True) 
costs_df.head()

In [ ]:
costs_df = costs_df.set_index("state")
costs_df.head()

In [ ]:
costs_df['total_cost'] = costs_df['total_cost'].str.replace(',', '')
costs_df['cost_per_capita'] = costs_df['cost_per_capita'].str.replace(',', '')
costs_df.head()

In [ ]:
csv_file = "./Data/Intensity of Binge Drinking.csv"
intensity_df = pd.read_csv(csv_file)
intensity_df.head()

In [ ]:
# rename columns in intensity_df, the website, https://www.cdc.gov/alcohol/data-stats.htm, says this is not a percentage but
# the average largest number of drinks consumed by binge drinkers on any occasion
intensity_df.rename(columns = {'State':'state', 'Percentage':'average_num_of_drinks'}, inplace = True) 
intensity_df.head()

In [ ]:
intensity_df = intensity_df.set_index("state")
intensity_df.head()

In [ ]:
csv_file = "./Data/Prevalence of Binge Drinking.csv"
prevalence_df = pd.read_csv(csv_file)
prevalence_df.head()

In [ ]:
prevalence_df.rename(columns = {'State':'state', 'Percentage':'percentage'}, inplace = True) 
prevalence_df.head()

In [ ]:
selected_column = prevalence_df[["state"]]

In [ ]:
prevalence_df = prevalence_df.set_index("state")
prevalence_df.head()

In [ ]:
csv_file = "./Data/Health.csv"
health_df = pd.read_csv(csv_file)
health_df.head()

In [ ]:
health_df = health_df.drop([0]).reset_index(drop = True)
health_df.head()

In [ ]:
health_df = health_df.join(selected_column)

In [ ]:
health_df.head()

In [ ]:
health_df = health_df.drop(columns=['Location', 'Male', 'Female', 'Footnotes'])
health_df.head() 

In [ ]:
health_df.rename(columns = {'All Adults':'health_percent'}, inplace = True) 
health_df.head()

In [ ]:
health_df['health_percent'] = health_df['health_percent'].str.replace('%', '')
health_df.head()


In [ ]:
health_df['health_percent'] = health_df['health_percent'].astype(float)
health_df.head()

In [ ]:
health_df = health_df.set_index("state")
health_df.head()

In [ ]:
conn = psycopg2.connect(database="Alcohol", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()

In [ ]:
schema_sql = open('./sql/schema.sql','r')
cursor.execute(schema_sql.read())
print("Database Schema Loaded Successfully (Tables Created)........")

In [ ]:
schema_sql.close()

In [ ]:
#database connection
connection_string ="postgres:Shjais2014@localhost:5432/Alcohol"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#confirm tables
engine.table_names()

In [ ]:
### Load DataFrames into database
breweries_df.to_sql(name='breweries', con=engine, if_exists='append', index = True)

In [ ]:
costs_df.to_sql(name='costs', con=engine, if_exists='append', index = True)

In [ ]:
intensity_df.to_sql(name='intensity', con=engine, if_exists='append', index=True)

In [ ]:
prevalence_df.to_sql(name='prevalence', con=engine, if_exists='append', index = True)

In [ ]:
health_df.to_sql(name='health', con=engine, if_exists='append', index=True)

In [ ]:
join_sql = open('sql/schema_2.sql','r')
alcohol_df = pd.read_sql_query(join_sql.read(), con=engine)
alcohol_df

In [ ]:
alcohol_df.to_csv('./Data/merged_data.csv')

In [ ]:
#Close File after use
join_sql.close()